In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("/data/credit-score-classification/train.csv")

In [3]:
df = df.drop(columns=["ID","Customer_ID","Name","Month","SSN"])

In [4]:
df = df.dropna()

In [5]:
df

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
6,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,...,Good,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
8,28_,_______,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,...,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
9,28,Teacher,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,7,...,Good,605.03,38.550848,26 Years and 8 Months,No,18.816215,40.39123782853101,High_spent_Large_value_payments,484.5912142650067,Good
10,28,Teacher,34847.84_,3037.986667,2,1385,6,1,Credit-Builder Loan,3,...,_,605.03,33.224951,26 Years and 9 Months,No,18.816215,58.51597569589465,High_spent_Large_value_payments,466.46647639764313,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,25,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",20,...,_,502.38,39.323569,31 Years and 5 Months,No,35.104023,140.58140274528395,High_spent_Medium_value_payments,410.256158,Poor
99995,25,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",23,...,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.866228,Poor
99996,25,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",18,...,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.65161,Poor
99997,25,Mechanic,39628.99,3359.415833,4,6,5729,2,"Auto Loan, and Student Loan",27,...,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.809083,Poor


In [6]:
df.shape

(58933, 23)

In [7]:
cat = df.select_dtypes("object")
num = df.select_dtypes(["int","float"])
check = cat.drop(columns=["Type_of_Loan","Occupation","Credit_Mix","Credit_History_Age","Payment_of_Min_Amount","Payment_Behaviour","Credit_Score"])
cat = cat.drop(columns=check.columns)

In [8]:

for x in check.columns:
    to_remove = []
    for y in check[x].values:
        try:
            y = float(y)
        except:
            to_remove.append(y)
    df = df[~df[x].isin(to_remove)]
    df[x] = df[x].astype("float")



In [9]:
cat = df.select_dtypes("object")
num = df.select_dtypes(["int","float"])

In [10]:
for x in cat.columns:
    df = df[df[x] != "_"]

df = df[df["Occupation"] != "_______"]
cat = df.select_dtypes("object")
num = df.select_dtypes(["int","float"])

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for x in cat.columns:
    df[x] = le.fit_transform(df[x])

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
x = df.drop(columns="Credit_Score")
y = df.Credit_Score
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [13]:
df.shape

(33023, 23)

In [14]:
RFC = RandomForestClassifier(n_estimators = 3000)
RFC.fit(x_train,y_train)

RandomForestClassifier(n_estimators=3000)

In [15]:
predict = RFC.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test,predict))

              precision    recall  f1-score   support

           0       0.72      0.70      0.71      1087
           1       0.77      0.78      0.78      2022
           2       0.80      0.80      0.80      3496

    accuracy                           0.78      6605
   macro avg       0.76      0.76      0.76      6605
weighted avg       0.78      0.78      0.78      6605



In [16]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)
predict_lr = lr.predict(x_test)
print(classification_report(y_test,predict_lr))

              precision    recall  f1-score   support

           0       0.22      0.00      0.00      1087
           1       0.53      0.46      0.49      2022
           2       0.56      0.78      0.65      3496

    accuracy                           0.55      6605
   macro avg       0.44      0.41      0.38      6605
weighted avg       0.50      0.55      0.50      6605



In [17]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier(n_estimators = 4000)
GBC.fit(x_train,y_train)
predict2 = GBC.predict(x_test)
print(classification_report(y_test,predict2))

              precision    recall  f1-score   support

           0       0.68      0.66      0.67      1087
           1       0.75      0.73      0.74      2022
           2       0.77      0.79      0.78      3496

    accuracy                           0.75      6605
   macro avg       0.73      0.73      0.73      6605
weighted avg       0.75      0.75      0.75      6605

